In [2]:
import sqlite3
import pandas as pd
import re

connection = sqlite3.connect('german_credit.db')
cursor = connection.cursor()

def select(query):
    try:
        return pd.read_sql_query(query, connection)
    except:
        cursor.executescript(query)
        connection.commit()

In [30]:
pd.read_csv('../german_credit_augmented_transactions.csv').to_sql('german_credit_transactions', connection, index=False, if_exists='replace')
connection.commit()

In [14]:
query = r'''
    WITH RECURSIVE t(n) AS (
    SELECT 1
    UNION ALL
    SELECT n + 1
    FROM t
    WHERE n < 10
    )
    SELECT n 
    FROM t
'''
select(query)

,n
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [31]:
# day counter
query = r'''
    WITH RECURSIVE dates(date) AS (
    VALUES('2024-01-01')
    UNION ALL
    SELECT date(date, '+1 day')
    FROM dates
    WHERE date < '2024-01-10'
    )
    SELECT date 
    FROM dates
'''
select(query)

,date
0,2024-01-01
1,2024-01-02
2,2024-01-03
3,2024-01-04
4,2024-01-05
5,2024-01-06
6,2024-01-07
7,2024-01-08
8,2024-01-09
9,2024-01-10


In [26]:
# month counter
query = r'''
    WITH RECURSIVE dates(date) AS (
    VALUES('2024-01-01')
    UNION ALL
    SELECT date(date, '+1 month')
    FROM dates
    WHERE date < '2024-12-01'
    )
    --SELECT strftime('%Y-%m', date) AS month
    SELECT date AS month
    FROM dates
'''
select(query)

,month
0,2024-01-01
1,2024-02-01
2,2024-03-01
3,2024-04-01
4,2024-05-01
5,2024-06-01
6,2024-07-01
7,2024-08-01
8,2024-09-01
9,2024-10-01


In [21]:
# year counter
query = r'''
    WITH RECURSIVE dates(date) AS (
    VALUES('2020-01-01')
    UNION ALL
    SELECT date(date, '+1 year')
    FROM dates
    WHERE date < '2024-01-01'
    )
    SELECT strftime('%Y', date) AS year
    FROM dates
'''
select(query)

,year
0,2020
1,2021
2,2022
3,2023
4,2024


In [32]:
query = r'''
    SELECT * 
    FROM german_credit_transactions
    LIMIT 10
'''
select(query)

,dt,client_id,amount
0,2008-04-06 11:54:47,950,161.38
1,2007-07-28 00:00:19,418,35.34
2,2008-03-14 20:43:54,131,146.50
3,2007-12-18 13:03:24,353,119.21
4,2007-11-09 05:18:30,849,105.24
5,2008-01-24 23:02:24,999,164.94
6,2007-12-24 16:28:30,938,1044.00
7,2007-05-07 17:46:15,568,98.39
8,2008-05-31 13:02:21,365,10049.10
9,2008-05-09 10:01:21,961,182.82


In [53]:
month_start_transactions = r'''
    SELECT date(MIN(dt), 'start of month') AS month
    FROM german_credit_transactions
'''
month_end_transactions = r'''
    SELECT date(MAX(dt), 'start of month') AS month
    FROM german_credit_transactions
'''


In [61]:
# All months
get_all_dates_query = rf'''
WITH dates(date) AS (
    VALUES(({month_start_transactions}))
    UNION ALL
    SELECT date(date, '+1 month')
    FROM dates
    WHERE date < ({month_end_transactions})
)
SELECT date
FROM dates
'''
select(get_all_dates_query)

,date
0,2007-05-01
1,2007-06-01
2,2007-07-01
3,2007-08-01
4,2007-09-01
5,2007-10-01
6,2007-11-01
7,2007-12-01
8,2008-01-01
9,2008-02-01


In [57]:
query = r'''
    SELECT * 
    FROM german_credit
    LIMIT 5
'''
select(query)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,None,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,None,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [60]:
# all users
get_all_clients_query = r'''
    SELECT DISTINCT client_id
    FROM german_credit
    ORDER BY client_id
'''
select(get_all_clients_query)

,client_id
0,0
1,1
2,2
3,3
4,4
...,...
995,995
996,996
997,997
998,998


In [67]:
# get all users + all months
query = rf'''
    WITH get_all_dates AS (
        {get_all_dates_query}
    ), get_all_clients AS (
        {get_all_clients_query}
    )
    SELECT 
        client_id AS client,
        date AS month
    FROM get_all_clients
    JOIN get_all_dates ON 1=1
    ORDER BY client, month
'''
select(query)

,client,month
0,0,2007-05-01
1,0,2007-06-01
2,0,2007-07-01
3,0,2007-08-01
4,0,2007-09-01
...,...,...
13995,999,2008-02-01
13996,999,2008-03-01
13997,999,2008-04-01
13998,999,2008-05-01


In [73]:
get_transactions_total = rf'''
    SELECT 
        client_id AS client,
        date(dt, 'start of month') AS month,
        SUM(amount) AS amount
    FROM german_credit_transactions
    GROUP BY client, month
    ORDER BY client, month
'''
select(get_transactions_total)

,client,month,amount
0,100,2007-08-01,304.43
1,100,2007-10-01,103.81
2,100,2008-01-01,140.99
3,100,2008-04-01,160.36
4,100,2008-05-01,51.47
...,...,...,...
3551,999,2007-06-01,1417.46
3552,999,2007-11-01,1022.32
3553,999,2008-01-01,-263.69
3554,999,2008-04-01,29.69


In [77]:
query = rf'''
    WITH get_all_dates AS (
        {get_all_dates_query}
    ), get_all_clients AS (
        {get_all_clients_query}
    ), get_transactions_total AS (
        {get_transactions_total}
    )
    SELECT 
        client_id AS client,
        date AS month, 
        COALESCE(amount, 0) AS total_month_trans
    FROM get_all_clients AS ci
        JOIN get_all_dates AS dt ON 1=1
        LEFT JOIN get_transactions_total AS tr ON ci.client_id = tr.client AND dt.date = tr.month
    ORDER BY client, month
'''
select(query)

,client,month,total_month_trans
0,0,2007-05-01,0.00
1,0,2007-06-01,0.00
2,0,2007-07-01,0.00
3,0,2007-08-01,0.00
4,0,2007-09-01,0.00
...,...,...,...
13995,999,2008-02-01,0.00
13996,999,2008-03-01,0.00
13997,999,2008-04-01,29.69
13998,999,2008-05-01,0.00


In [90]:
month_start_transactions = r'''
    SELECT date(MIN(dt), 'start of month') AS month
    FROM german_credit_transactions
'''
month_end_transactions = r'''
    SELECT date(MAX(dt), 'start of month') AS month
    FROM german_credit_transactions
'''
query = rf'''
    WITH get_all_clients AS ( 
        SELECT 
            DISTINCT client_id
        FROM german_credit
        ORDER BY client_id
    ), dates(date) AS (
        VALUES(({month_start_transactions}))
        UNION ALL
        SELECT 
            date(date, '+1 month')
        FROM dates
        WHERE date < ({month_end_transactions})
    ), get_months AS (
        SELECT 
            date AS month
        FROM dates
    ), get_clients_n_months AS (
    SELECT 
        ci.client_id AS client,
        dt.month AS month
    FROM get_all_clients AS ci
        JOIN get_months AS dt ON 1=1
    ), get_total_transactions AS (
    SELECT 
        gt.client_id AS client,
        DATE(gt.dt, 'start of month') AS month,
        SUM(gt.amount) AS total
    FROM german_credit_transactions AS gt
    GROUP BY client, month
    )
    SELECT 
        dtc.client AS client_id,
        dtc.month AS month,
        COALESCE(tr.total, 0) AS total_amount
    FROM get_clients_n_months AS dtc
        LEFT JOIN get_total_transactions AS tr 
            ON tr.client = dtc.client AND tr.month = dtc.month
    ORDER BY client_id, month
'''
select(query)

,client_id,month,total_amount
0,0,2007-05-01,0.00
1,0,2007-06-01,0.00
2,0,2007-07-01,0.00
3,0,2007-08-01,0.00
4,0,2007-09-01,0.00
...,...,...,...
13995,999,2008-02-01,0.00
13996,999,2008-03-01,0.00
13997,999,2008-04-01,29.69
13998,999,2008-05-01,0.00
